In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
from src.preprocessing.preprocessing import load_and_split_data, NUM_CLASSES 
from src.models.model_definition import create_transfer_model

In [4]:
BATCH_SIZE = 32
train_ds, val_ds, test_ds, ds_info = load_and_split_data(batch_size=BATCH_SIZE)

In [5]:
model = create_transfer_model(input_shape=(224, 224, 3))
model.summary()

Model: "EfficientNet_Transfer_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 4054695 (15.47 MB)
Trainable params: 5124 (20.02 KB)
Non-trainable params: 4049571 (15.45 MB)
_______________________________________________________________

In [6]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,        
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath='models/best_efficientnet_4class.h5', 
    monitor='val_accuracy', 
    mode='max', 
    save_best_only=True
)

In [7]:
history = model.fit(
    train_ds,
    epochs=25,
    validation_data=val_ds,
    callbacks=[early_stopping, model_checkpoint]
)

model.save('models/final_efficientnet_4class.h5')

Epoch 1/25



     65/Unknown - 41s 385ms/step - loss: 0.6929 - accuracy: 0.7578

C:\Users\socik\Documents\food-recognition\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


65/65 [==============================] - 50s 535ms/step - loss: 0.6929 - accuracy: 0.7578 - val_loss: 0.3402 - val_accuracy: 0.9116
Epoch 2/25
65/65 [==============================] - 41s 491ms/step - loss: 0.3492 - accuracy: 0.8950 - val_loss: 0.2520 - val_accuracy: 0.9221
Epoch 3/25
65/65 [==============================] - 42s 494ms/step - loss: 0.2760 - accuracy: 0.9095 - val_loss: 0.2192 - val_accuracy: 0.9158
Epoch 4/25
65/65 [==============================] - 41s 486ms/step - loss: 0.2554 - accuracy: 0.9148 - val_loss: 0.2027 - val_accuracy: 0.9242
Epoch 5/25
65/65 [==============================] - 41s 488ms/step - loss: 0.2294 - accuracy: 0.9182 - val_loss: 0.1827 - val_accuracy: 0.9284
Epoch 6/25
65/65 [==============================] - 42s 494ms/step - loss: 0.2160 - accuracy: 0.9287 - val_loss: 0.1749 - val_accuracy: 0.9305
Epoch 7/25
65/65 [==============================] - 41s 487ms/step - loss: 0.2009 - accuracy: 0.9326 - val_loss: 0.1752 - val_accuracy: 0.9347
Epoch 8/25

In [8]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Trening Accuracy')
    plt.plot(epochs_range, val_acc, label='Walidacja Accuracy')
    plt.legend(loc='lower right')
    plt.title('Dokładność (Accuracy)')
    plt.grid(True)

    # Wykres Straty (Loss)
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Trening Loss')
    plt.plot(epochs_range, val_loss, label='Walidacja Loss')
    plt.legend(loc='upper right')
    plt.title('Strata (Loss)')
    plt.grid(True)
    
    plt.show()